### _해당 파일은 텍스트분석을 도와주는 함수들을 정리한 파일이다_

## 기본 패키지들

In [ ]:
#코랩에서 konlpy 설치하는 방법
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
#base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#정규표현식
import re
import warnings
warnings.filterwarnings('ignore')

#불용어
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

#토큰화
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer

from tensorflow.keras.preprocessing.text import text_to_word_sequence

#워드클라우드
from wordcloud import WordCloud
from konlpy.tag import Twitter
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wips\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wips\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wips\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wips\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wips\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
path = 'C:/Users/wips/Desktop/업무폴더/1주차업무/'
data =pd.read_csv(path+'facebook.csv')
data.head()

,num,data,ryu,text
0,1,2021.09.09,"09, 41, 42, 45",Application programming interface (API) for co...
1,2,2021.09.09,"09, 38, 41, 42",Computer hardware; software development tools;...
2,3,2020.06.18,9,Computer hardware and downloadable software fo...
3,4,2020.06.12,"09, 35, 41, 42, 45",Downloadable application programming interface...
4,5,2020.06.11,"09, 35, 41, 42, 45",Application programming interface (API) for co...


## 전처리 함수

### punctuations

In [ ]:
# 다른 문자들의 존재 여부 확인 [^안에 든 문자들 뺀]
def show_not_alphanum(text):
    return re.findall("[^0-9a-zA-Z‘’\'\- \,\. ]",text)

In [ ]:
# 주의할 연결 문자   
text.apply(lambda x : re.findall("\w{1,}\_\w{1,}",x)) # 언더바(_)
text.apply(lambda x : re.findall("\w{1,}\-\w{1,}",x)) # 하이픈(-)
text.apply(lambda x : re.findall("\w{1,}\'\w{1,}",x)) # appostrophe(') 
text.apply(lambda x : re.findall("\w{1,}\.\w{1,}",x)) # 간혹 마침표

In [ ]:
# 다른 문자들은 모두 없애줘
def only_alphanum(text):
    return re.sub("[^0-9a-zA-Z\- ]",'',text)

### tokenization

In [ ]:
WordPunctTokenizer().tokenize(text)
TreebankWordTokenizer().tokenize(text)

### capitals

In [ ]:
#대문자 확인
def check_caps(all_list):
    empty = []
    for i in all_list:
        if i.isupper() == True:
            empty.append(i)
    return empty

In [ ]:
# 문제 없는 애들은 모두 소문자로 변경
data = [i for i in data if i not in set(capitals)]
data = [x.lower() for x in data]

### short words

In [ ]:
data = [i for i in data if len(i)>1]

### stopwords

In [ ]:
stop_words_list = stopwords.words('english');len(stop_words_list)
tok_2021 = [i for i in tok_2021 if i not in stop_words_list]

### Lenmmatization

In [ ]:
lemmatizer = WordNetLemmatizer()
lemma_toks = [lemmatizer.lemmatize(x) for x in tok_2021]
print(len(set(tok_2021)),len(set(lemma_toks))) #단어가 줄긴했음

### 특정 품사만 추출

In [ ]:
adj_list=[]
for j in docs:
  bin_list2=[]
  for word in nltk.tag.pos_tag(j):
    if word[1] in [ 'JJ', 'JJR', 'JJS']: #형용사
      bin_list2.append(word[0])
#    bin_list2 = [i for i in bin_list2 if i not in ['']] 
  adj_list.append(bin_list2)
len(adj_list)

### auto-process

In [ ]:
#데이터프레임 내 텍스트를 토큰화하는 함수

def all_process(datas):
    
    datas.text = datas.text.apply(only_alphanum)
    tokens = datas.text.apply(lambda x : tokenizer.tokenize(x))
    tok=[]
    
    for i in tokens.index:
      tok.extend(tokens[i])

    capitals = list(check_caps(tok))
    tok = [i for i in tok if i not in set(capitals)]
    tok = [x.lower() for x in tok]
    tok.extend(capitals)

    tok = [i for i in tok if len(i)>1]
    tok = [i for i in tok if i not in stop_words_list]

    lemmatizer = WordNetLemmatizer()
    lemma_toks = [lemmatizer.lemmatize(x) for x in tok]

    return lemma_toks

### technics

In [ ]:
# make yearly data

years =list(data.date.str[:4].unique()) #2005년부터 2021년까지
#years.sort(reverse=True)

data_list =['data_' + i for i in years] #데이터 명 생성

#연도 별 dataframe
for i,j in enumerate(years):
  locals()[data_list[i]] = data.loc[data['date'].apply(lambda x : x.startswith(j)),]

In [ ]:
# 모든 문서에서 자주나오는 단어는 문서 구분에서는 필요 없다.
# 이를 다시 반영하여 함수 생성
def all_process2(datas):
    datas.text = datas.text.apply(only_alphanum)

    tokens = datas.text.apply(lambda x : tokenizer.tokenize(x))
    tok=[]
    for i in tokens.index:
      tok.extend(tokens[i])

    capitals = list(check_caps(tok))
    tok = [i for i in tok if i not in set(capitals)]
    tok = [x.lower() for x in tok]
    tok.extend(capitals)

    tok = [i for i in tok if len(i)>1]
    tok = [i for i in tok if i not in stop_words_list]

    lemmatizer = WordNetLemmatizer()
    lemma_toks = [lemmatizer.lemmatize(x) for x in tok]

    lemma_toks = [i for i in lemma_toks if i not in most_frequency_words]

    return lemma_toks


for i,j in enumerate(token_list):
  locals()[j] = all_process2(data.loc[data['date'].apply(lambda x : x.startswith(years[i])),])

### DTM

In [ ]:
from math import log

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

### TF-IDF

In [ ]:
#N = 총 문서의 수
#docs = [문서들]
#vocab = list(words)

def tf_idf_frame(N,docs,vocab):
    result = []
    for i in range(N):
      result.append([])
      d = docs[i]
      for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

    tfidf_ = pd.DataFrame(result, columns = vocab)
    return(tfidf_)

## 시각화

### 워드클라우드

In [ ]:
def cloud_making(data):
    wc = WordCloud(background_color="white", max_font_size=60)
    cloud = wc.generate_from_frequencies(dict(data))
    
    plt.figure(figsize=(10, 8))
    plt.axis('off')
    plt.imshow(cloud)
    plt.show()

### 예쁜 그래프 그리기

In [ ]:
def main():
	#### 1. bar plot으로 나타낼 데이터 입력
	models = ['unique_words']
	yticks = list(cnt_all_df.index)
	data = {'unique_words' :list(cnt_all_df.unique_words)}
	
	#### 2. matplotlib의 figure 및 axis 설정
	fig, ax = plt.subplots(1,1,figsize=(7,10)) # 1x1 figure matrix 생성, 가로(14인치)x세로(10인치) 크기지정
	height = 0.15
	#ax.axvline(200, c='r', ls='--') 내가 원하는 틱에 빨간선 긋는 코드
	#### 3. bar 그리기
	for i, model in enumerate(models):
		pos = compute_pos(yticks, height, i, models)
		bar = ax.barh(pos, data[model], height=height*0.95, label=model)
		present_width(ax, bar) # bar너비 출력
	
	#### 4. x축 세부설정
	ax.set_xlim([내가원하는범위]) 
	ax.set_xticks([내가원하는틱])
	ax.xaxis.set_tick_params(labelsize=10)
	ax.set_xlabel('x-name', fontsize=14)
	
	#### 5. y축 세부설정
	ax.set_yticks(range(len(yticks)))
	ax.set_yticklabels(yticks, fontsize=10)	
	ax.set_ylabel('y-name', fontsize=10)
	
	#### 6. 범례 나타내기
	box = ax.get_position() # 범례를 그래프상자 밖에 그리기위해 상자크기를 조절
	ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
	ax.legend(loc='upper right',  shadow=True, ncol=1)
	
	#### 7. 보조선(눈금선) 나타내기
	ax.set_axisbelow(True)
	ax.xaxis.grid(True, color='gray', linestyle='dashed', linewidth=0.5)
	
	#### 8. 그래프 저장하고 출력하기
	#plt.savefig('ex_barhplot.png', format='png', dpi=300)

	plt.show()
	
def compute_pos(yticks, height, i, models):
  index = np.arange(len(yticks))
  n = len(models)
  correction = i - 0.5*(n-1)
  return index + height * correction

def present_width(ax, bar):
	for rect in bar:
		witdh = rect.get_width()
		posx = witdh*1.01
		posy = rect.get_y()+rect.get_height()*0.5
		ax.text(posx, posy, witdh, rotation=0, ha='left', va='center')

# 이거 코드 돌리면 바로 실행됨!        
if __name__=='__main__':
	main()
 


### 데이터프레임 기반 깔끔한 그래프

In [ ]:
#bar
def bar_graph():
    fig2, ax2 = plt.subplots()
    df.plot.bar(x='index',alpha=0.5,figsize=(10,10),ax=ax2)
    plt.xticks(rotation=0,fontsize=13)
    plt.title('VR vs SNS by rate(%)',fontsize=15)
    plt.xlabel([],fontsize=0)
    # ax2.set_yticklabels([]) 틱 없애줌
    plt.tick_params(axis='both',which='both',left=False)
    plt.show()

In [ ]:
#line
def line_graph():
    fig2, ax2 = plt.subplots()
    vr_sns_df.VR.plot(x=vr_sns_df['index'],alpha=0.3,figsize=(10,5),ax=ax2, linewidth=3)
    vr_sns_df.SNS.plot(x=vr_sns_df['index'],alpha=0.3,figsize=(10,5),ax=ax2, linewidth=3)
    ax2.set_yticklabels([])
    ax2.set_xticklabels(['0']+list(vr_sns_df['index']))
    plt.tick_params(axis='both',which='both',left=False)
    title = 'VR vs SNS by rate(%)'
    ax2.set_title(title,fontsize=15)
    ax2.legend(prop={'size':10},loc=2)
    plt.show()